# Use Case: A Knowledge Graph About Alocholic Beverages
We are going to build a small KG about alcoholoc beverages by extracting from Wikidata the subgraph that relates to alcoholic beverages (https://www.wikidata.org/wiki/Q154)

In [1]:
import sys  
sys.path.insert(0, 'tutorial')
from tutorial_setup import *

ALIAS: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/aliases.en.tsv.gz"
ALL: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/all.tsv.gz"
CLAIMS: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/claims.tsv.gz"
DESCRIPTION: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/descriptions.en.tsv.gz"
EXAMPLES_DIR: "/Users/pedroszekely/Documents/GitHub/kgtk/examples"
GE: "/Users/pedroszekely/Downloads/kgtk-tutorial/temp/graph-embedding"
ISA: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/derived.isa.tsv.gz"
ITEM: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/claims.wikibase-item.tsv.gz"
KGTK_PATH: "/Users/pedroszekely/Documents/GitHub/kgtk"
LABEL: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/labels.en.tsv.gz"
OUT: "/Users/pedroszekely/Downloads/kgtk-tutorial/output"
P279: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/derived.P279.tsv.gz"
P279STAR: "/Users/pedroszekely/Downloads/kgtk-tutorial/miniwikidata/de

In [2]:
%cd {output_path}

/Users/pedroszekely/Downloads/kgtk-tutorial


In [3]:
!mkdir -p {output_folder}
!mkdir -p {temp_folder}

### Step 1: create a list of all descendants of `alcoholic beverage` (https://www.wikidata.org/wiki/Q154)
Here is some of the information about `Q154` in Wikidata:

In [4]:
result = !$kypher -i claims -i labels \
--match 'claims: (n1:Q154)-[l {label:p}]->(n2), label: (n2)-[]->(n2_label), label: (p)-[]->(p_label)' \
--return 'n1 as node1, l.label as label, p_label as `label;label`, n2 as node2, n2_label as `node2;label`' \
--order-by 'l.label'


kgtk_to_dataframe(result)

,node1,label,label;label,node2,node2;label
0,Q154,P1151,'topic\\'s main Wikimedia portal'@en,Q61473108,'Portal:Alcoholic drinks'@en
1,Q154,P1343,'described by source'@en,Q1768721,'Gujin Tushu Jicheng'@en
2,Q154,P1424,'topic\\'s main template'@en,Q10557691,'Template:Infobox alcoholic beverage'@en
3,Q154,P1552,'has quality'@en,Q1517187,'bitterness'@en
4,Q154,P2354,'has list'@en,Q2468826,'list of alcoholic beverages'@en
5,Q154,P279,'subclass of'@en,Q40050,'drink'@en
6,Q154,P31,'instance of'@en,Q187661,'carcinogen'@en
7,Q154,P31,'instance of'@en,Q8386,'drug'@en
8,Q154,P452,'industry'@en,Q3150593,'alcohol industry'@en
9,Q154,P461,'opposite of'@en,Q2647467,'non-alcoholic beverage'@en


Wikidata uses two properties to organize entities in a hierarchy: the `instance of` property (`P31`) and the `subclass of` (`P279`) property. In many cases, the distinction between instance of and subclass of is subtle, and we find many situations in Wikidata where either one or the other is used to organize hierarchies. For this reason, we created a new property called `isa` that contains the union of `P31` and `P279` and stored in the file `derived.isa.tsv`

In [5]:
lines = !zcat < "$ISA" | head -5
kgtk_to_dataframe(lines)

,node1,label,node2
0,P10,isa,Q18610173
1,P1000,isa,Q18608871
2,P1001,isa,Q15720608
3,P1001,isa,Q22984026


To get all the alcoholic beverages, we need to get all entities that are `isa` of alcoholic beverage (`Q154`) or that are `isa` of any descendant of `Q154` in the `subclass of` (`P279`) hierarchy. The length of the chain of `P279` edges can be arbitrarily long. To support this use case, KGTK offers the `derived.P279star.tsv` file that contains edges `n1/P279star/n2` if `n1` is a descendant of `n2` on chains of `P279` edges, includiing chains of zero length (`n1/P279star/n1`).

In [6]:
lines = !zcat < "$P279STAR" | head -5 
kgtk_to_dataframe(lines)

,node1,label,node2,id
0,Q100000030,P279star,Q100000030,Q100000030-P279star-Q100000030-0000
1,Q100000030,P279star,Q1357761,Q100000030-P279star-Q1357761-0000
2,Q100000030,P279star,Q14745,Q100000030-P279star-Q14745-0000
3,Q100000030,P279star,Q14748,Q100000030-P279star-Q14748-0000


To get all alcoholic beverages, we need to find all nodes `n1` that are connected to `Q154` with an `isa` edge and a chain of `P279` edges:

In [7]:
!$kypher -i "$ISA" --as "isa" -i "$P279STAR" --as "p279star" -i labels \
--match 'isa: (n1)-[]->(n2), star: (n2)-[]->(n3:Q154), label: (n1)-[]->(n1l)' \
--return 'n1 as node1, n1l as `node1;label`, n3 as node2, "isastar" as label' \
-o "$TEMP"/Q154.descendant.tsv

Here is a sample of alcoholic beverages in Wikidata

In [8]:
lines = !cat "$TEMP"/Q154.descendant.tsv 
kgtk_to_dataframe(lines)

,node1,node1;label,node2,label
0,Q1350656,'corn whiskey'@en,Q154,isastar
1,Q20713240,'Buckwheat whisky'@en,Q154,isastar
2,Q2535077,'rye whiskey'@en,Q154,isastar
3,Q536976,'Canadian whisky'@en,Q154,isastar
4,Q7991845,'wheat whiskey'@en,Q154,isastar
...,...,...,...,...
3346,Q7719471,'The Botanist'@en,Q154,isastar
3347,Q187155,'Tanqueray'@en,Q154,isastar
3348,Q62076228,'dry gin'@en,Q154,isastar
3349,Q7085234,'Old Tom Gin'@en,Q154,isastar


The computation of `Q154.descendant.tsv` can be implemented in SPARQL using the common `P31/P279*` graph pattern, but the query will time out if the result size is large. For example, the query will time out when requesting all descendants of chemical compounds, as there are over one million chemical compounds in Wikidata. The query can be easily done in KGTK.

### Step 2: get the incoming and outgoing edges
We want out graph to have the neighbors of all alcoholic beverages, so we need to get the incoming and outgoing edges.

The following query gets the **outgoing** edges.

In [9]:
!$kypher -i claims -i "$TEMP"/Q154.descendant.tsv \
--match 'Q154: (n1)-[]->(), claims: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.node1.tsv.gz

We see that we are getting several properties for our items:

In [10]:
lines = !zcat < "$TEMP"/Q154.node1.tsv.gz | head 
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,Q1000737-P1435-Q17297633-53903946-0,Q1000737,P1435,Q17297633
1,Q1000737-P1454-Q460178-8ad4931b-0,Q1000737,P1454,Q460178
2,Q1000737-P159-Q16003-31e24011-0,Q1000737,P159,Q16003
3,Q1000737-P17-Q183-24107fe2-0,Q1000737,P17,Q183
4,Q1000737-P18-147fc9-667304f8-0,Q1000737,P18,"""Marthabräuhalle 2011-04-03.jpg"""
5,Q1000737-P31-Q131734-f97bd6f6-0,Q1000737,P31,Q131734
6,Q1000737-P31-Q15075508-a4c83928-0,Q1000737,P31,Q15075508
7,Q1000737-P373-689157-3110aade-0,Q1000737,P373,"""Marthabräu"""
8,Q1000737-P452-Q869095-f5d8e7a2-0,Q1000737,P452,Q869095


Now get the **incoming** edges:

In [11]:
!$kypher -i claims -i "$TEMP"/Q154.descendant.tsv \
--match 'Q154: (n1)-[]->(), claims: (n3)-[l]->(n1)' \
--return 'distinct l as id, n3 as node1, l.label as label, n1 as node2' \
-o "$TEMP"/Q154.node2.tsv.gz

Here is a sample of the edges we are getting

In [12]:
lines = !zcat < "$TEMP"/Q154.node2.tsv.gz | head
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,Q1350656-P279-Q1007164-7e3ecba9-0,Q1350656,P279,Q1007164
1,Q20713240-P279-Q1007164-b3112260-0,Q20713240,P279,Q1007164
2,Q2535077-P279-Q1007164-b2d3684b-0,Q2535077,P279,Q1007164
3,Q536976-P279-Q1007164-8bf7467b-0,Q536976,P279,Q1007164
4,Q7991845-P279-Q1007164-18bc383a-0,Q7991845,P279,Q1007164
5,Q10337004-P186-Q10210-c56dd7ce-0,Q10337004,P186,Q10210
6,Q10429117-P31-Q10210-d342f061-0,Q10429117,P31,Q10210
7,Q1051699-P279-Q10210-65d32c67-0,Q1051699,P279,Q10210
8,Q1058259-P279-Q10210-e204554a-0,Q1058259,P279,Q10210


Concatenate the incoming and outgoing edges to put them in a single file:

In [13]:
!$kgtk cat -i "$TEMP"/Q154.node1.tsv.gz -i "$TEMP"/Q154.node2.tsv.gz -o "$TEMP"/Q154.claims.tsv.gz

See how many edges we have:

In [14]:
!zcat < "$TEMP"/Q154.claims.tsv.gz | wc

   31945  131399 1801186


Summary of where we are:
- Computed the list of q-nodes below alcoholic beverage
- Found all incoming and outgoing edges to these q-nodes; for the new q-nodes we bring in, we have no information, we only have the q-node

We have the q-nodes connected to alcoholic beverages, but so far we don't have the edges of those q-nodes. We need to go one hop out from the q-nodes that we have. We run a query to go one hop out from any qnode in `Q154.claims.tsv` which will use all the q-nodes in our graph, including the alcoholic beverages for which we already got outgoing edges; no harm done, as we can eliminate duplicated later.

In [15]:
!$kypher -i claims -i "$TEMP"/Q154.claims.tsv.gz \
--match 'Q154: ()-[]->(n1), claims: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.hop.out.tsv.gz

For sanity check, let's take a peek:

In [16]:
lines = !zcat < "$TEMP"/Q154.hop.out.tsv.gz | head 
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,Q1000-P1014-693343-9664fa33-0,Q1000,P1014,"""300262648"""
1,Q1000-P1036-9bef62-f77ac5cf-0,Q1000,P1036,"""2--6721"""
2,Q1000-P1081-0d345f-3a33abf5-0,Q1000,P1081,+0.641
3,Q1000-P1081-0d345f-6da37c02-0,Q1000,P1081,+0.641
4,Q1000-P1081-1100e3-c7631769-0,Q1000,P1081,+0.624
5,Q1000-P1081-1ada51-7c71c229-0,Q1000,P1081,+0.639
6,Q1000-P1081-345681-88a99cab-0,Q1000,P1081,+0.702
7,Q1000-P1081-347db1-da0e5e03-0,Q1000,P1081,+0.637
8,Q1000-P1081-419245-b03a8b59-0,Q1000,P1081,+0.647


Let's consolidate our edge files into one larger file. We use compact to remove duplicates and sort to keep edges for the same subject together:

In [17]:
!$kgtk cat -i "$TEMP"/Q154.claims.tsv.gz -i "$TEMP"/Q154.hop.out.tsv.gz \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.1.tsv.gz

See how many edges we have:

In [18]:
!zcat < "$TEMP"/Q154.edges.1.tsv.gz | wc

  159073  655180 8549211


Take a peek:

In [19]:
lines = !zcat < "$TEMP"/Q154.edges.1.tsv.gz | head 
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,P1389-P1855-Q1109662-9e2ef218-0,P1389,P1855,Q1109662
1,P1582-P1855-Q17329207-f4ef508d-0,P1582,P1855,Q17329207
2,P2581-P1855-Q7639844-08b3a4c7-0,P2581,P1855,Q7639844
3,P2665-P1855-Q1067702-402a80a9-0,P2665,P1855,Q1067702
4,P2665-P1855-Q170210-30d44f0b-0,P2665,P1855,Q170210
5,P5420-P1855-Q44-209cffb1-0,P5420,P1855,Q44
6,P5420-P1855-Q722338-73d7be75-0,P5420,P1855,Q722338
7,P5471-P1855-Q44-6c38949b-0,P5471,P1855,Q44
8,P6088-P1855-Q1543214-3d934541-0,P6088,P1855,Q1543214


### Step 3: get the ontology
Once we have all the alcoholic beverages, we want to get the upper ontology of all the classes used, so that every class in our KG has a path to the root of the ontology. For example, first go to `drink` (`Q40050`), then to `liquid` (`Q11435`), then `fluid` (`Q102205`) and so on until we reach `entity` (`Q35120`).

To do this, we need to get all the `isa` of all items in our graph, then get `P279star` so we get the list of all classes that these items descend from. Finally we need to get all the `P279` edges between them.

In [20]:
!$kypher -i "$TEMP"/Q154.edges.1.tsv.gz -i p279star -i isa \
--match 'Q154: (n1)-[]->(), isa: (n1)-[]->(n2), p279star: (n2)-[]->(class)' \
--return 'distinct class as node1' \
-o "$TEMP"/Q154.classes.tsv

See how many classes we have in the upper ontology for the entities in our graph:

In [21]:
!wc "$TEMP"/Q154.classes.tsv

    2791    2791   24573 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/Q154.classes.tsv


Check that `fluid` (`Q102205`) is listed in the classes:

In [22]:
!grep Q102205 "$TEMP"/Q154.classes.tsv

Q102205


Now use the `derived.P279.tsv` file to get the `P279` edges that connect a class to its superclass.

In [23]:
!$kypher -i "$TEMP"/Q154.classes.tsv -i "$P279" --as "p279" \
--match 'Q154: (class)-[]->(), p279: (class)-[l]->(super)' \
--return 'distinct l as id, class as node1, l.label as label, super as node2' \
-o "$TEMP"/Q154.P279.tsv

See how many `P279` edges are in the upper ontology; we will take care of potential duplicates at a final cleanup step:

In [24]:
!wc "$TEMP"/Q154.P279.tsv

    4428   17712  245148 /Users/pedroszekely/Downloads/kgtk-tutorial/temp/Q154.P279.tsv


We see several q-nodes below `entity` (`Q35120`), a good indication that we computed the upper ontology correctly:

In [25]:
!grep Q35120 "$TEMP"/Q154.P279.tsv | head -5 | column -t -s $'\t'

Q488383-P279-Q35120-5fad2ad7-0    Q488383    P279  Q35120
Q58415929-P279-Q35120-75659d0c-0  Q58415929  P279  Q35120
Q99527517-P279-Q35120-562a6511-0  Q99527517  P279  Q35120
Q16686448-P279-Q35120-674edbf9-0  Q16686448  P279  Q35120
Q23958946-P279-Q35120-70a9ed90-0  Q23958946  P279  Q35120


Let's consolidate the edges again:

In [26]:
!$kgtk cat -i "$TEMP"/Q154.edges.1.tsv.gz -i "$TEMP"/Q154.P279.tsv \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.2.tsv.gz

The number of edges is growing:

In [27]:
!zcat < "$TEMP"/Q154.edges.2.tsv.gz | wc

  162839  670244 8758729


Summary:
- We have the instances of alcoholic beverages
- We added incoming and outgoing edges
- For all the q-nodes in the previous step, we went one hop forward
- We got the upper ontology

### Step 4: get the edges for properties
The properties are also items in Wikidata, so let's collect them all and get their edges.

In [28]:
!$kypher -i "$TEMP"/Q154.edges.2.tsv.gz \
--match '()-[l {label: property}]->()' \
--return 'distinct property as node1' \
-o "$TEMP"/Q154.properties.tsv

In [29]:
!head "$TEMP"/Q154.properties.tsv | column -t -s $'\t'

node1
P10
P1001
P1004
P1005
P101
P1014
P1015
P1017
P1019


Let's get the edges of these properties:

In [30]:
!$kypher -i claims -i "$TEMP"/Q154.properties.tsv \
--match 'Q154: (p)-[]->(), claims: (p)-[l]->(n2)' \
--return 'distinct l as id, p as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.properties.edges.tsv

In the resulting file, `node1` is a property and now we have data about them:

In [31]:
lines = !head "$TEMP"/Q154.properties.edges.tsv 
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,P10-P1628-32b85d-7927ece6-0,P10,P1628,"""http://www.w3.org/2006/vcard/ns#Video"""
1,P10-P1628-acf60d-b8950832-0,P10,P1628,"""https://schema.org/video"""
2,P10-P1629-Q34508-bcc39400-0,P10,P1629,Q34508
3,P10-P1659-P1651-c4068028-0,P10,P1659,P1651
4,P10-P1659-P18-5e4b9c4f-0,P10,P1659,P18
5,P10-P1659-P4238-d21d1ac0-0,P10,P1659,P4238
6,P10-P1659-P51-86aca4c5-0,P10,P1659,P51
7,P10-P1855-Q7378-555592a4-0,P10,P1855,Q7378
8,P10-P2302-Q21502404-d012aef4-0,P10,P2302,Q21502404


Let's consolidate the edges again:

In [32]:
!$kgtk cat -i "$TEMP"/Q154.edges.2.tsv.gz -i "$TEMP"/Q154.properties.edges.tsv \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.3.tsv.gz

The number of edges grew a bit

In [33]:
!zcat < "$TEMP"/Q154.edges.3.tsv.gz | wc

  185031  763294 10102000


Summary:
- We have the instances of alcoholic beverages
- We added incoming and outgoing edges
- For all the q-nodes in the previous step, we went one hop forward
- We got the upper ontology
- And we have the edges on all the properties being used

### Step 5: get edges between any two q-nodes in our graph
As we added q-nodes to our graph, it is possible that there exist edges between these q-nodes that we didn't get when doing the one hop out. To ensure completeness, we get all edges from Wikidata between any two nodes in our Q154 graph. The following query ensures that we are not missing any adeges betwee the nodes we added to our KG. Note that this query is expensive as it needs to find edges between any two nodes. This can be done in kypher, but would be impssible in SPARQL as you would surely get a time-out.

In [34]:
!$kypher -i "$TEMP"/Q154.edges.3.tsv.gz -i "$ITEM" --as items \
--match 'Q154: (n1)-[]->(n2), item: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.edges.complete.tsv.gz

Concatenate the files again:

In [35]:
!$kgtk cat -i "$TEMP"/Q154.edges.3.tsv.gz -i "$TEMP"/Q154.edges.complete.tsv.gz \
/ compact \
/ sort2 \
-o "$TEMP"/Q154.edges.4.tsv.gz

We now have all the edges we want:

In [36]:
!zcat < "$TEMP"/Q154.edges.4.tsv.gz | wc

  185238  764122 10113585


### Step 6: get the labels, aliases and descriptions of all the items in our KG
Before we start, let's define an environment variable to hold the final edges file so that if we change our mind later, we can update it without having to change the commands below.

In [37]:
os.environ["Q154GRAPH"] = os.environ["TEMP"] + "/Q154.edges.4.tsv.gz"

In [38]:
!ls "$Q154GRAPH"

/Users/pedroszekely/Downloads/kgtk-tutorial/temp/Q154.edges.4.tsv.gz


#### Get the labels of the `node1` nodes

In [39]:
!$kypher -i "$Q154GRAPH" -i labels \
--match 'Q154: (n1)-[]->(), label: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.label.node1.tsv.gz

In [40]:
lines = !zcat < "$TEMP"/Q154.label.node1.tsv.gz | head 
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,P10-label-en,P10,label,'video'@en
1,P1001-label-en,P1001,label,'applies to jurisdiction'@en
2,P1004-label-en,P1004,label,'MusicBrainz place ID'@en
3,P1005-label-en,P1005,label,'Portuguese National Library ID'@en
4,P101-label-en,P101,label,'field of work'@en
5,P1014-label-en,P1014,label,'Art & Architecture Thesaurus ID'@en
6,P1015-label-en,P1015,label,'BIBSYS ID'@en
7,P1017-label-en,P1017,label,'Vatican Library ID'@en
8,P1019-label-en,P1019,label,'web feed URL'@en


Get the labels of the `node2` nodes

In [41]:
!$kypher -i "$Q154GRAPH" -i labels \
--match 'Q154: ()-[]->(n2), label: (n2)-[l]->(n3)' \
--return 'distinct l as id, n2 as node1, l.label as label, n3 as node2' \
-o "$TEMP"/Q154.label.node2.tsv.gz

Concatenate the two label files

In [42]:
!$kgtk cat -i "$TEMP"/Q154.label.node1.tsv.gz -i "$TEMP"/Q154.label.node2.tsv.gz \
-o "$TEMP"/labels.tsv.gz

#### Get the aliases of `node1` nodes

In [43]:
!$kypher -i "$Q154GRAPH" -i "$ALIAS" --as aliases \
--match 'Q154: (n1)-[]->(), alias: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.alias.node1.tsv.gz

Get the aliases of `node2` nodes

In [44]:
!$kypher -i "$Q154GRAPH" -i aliases \
--match 'Q154: ()-[]->(n2), alias: (n2)-[l]->(n3)' \
--return 'distinct l as id, n2 as node1, l.label as label, n3 as node2' \
-o "$TEMP"/Q154.alias.node2.tsv.gz

Concatenate the two alias files

In [45]:
!$kgtk cat -i "$TEMP"/Q154.alias.node1.tsv.gz -i "$TEMP"/Q154.alias.node2.tsv.gz \
-o "$TEMP"/alias.tsv.gz

#### Get the descriptions of `node1` nodes

In [46]:
!$kypher -i "$Q154GRAPH" -i "$DESCRIPTION" --as descriptions \
--match 'Q154: (n1)-[]->(), description: (n1)-[l]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.description.node1.tsv.gz

Get the descriptions of `node2` nodes

In [47]:
!$kypher -i "$Q154GRAPH" -i descriptions \
--match 'Q154: ()-[]->(n2), description: (n2)-[l]->(n3)' \
--return 'distinct l as id, n2 as node1, l.label as label, n3 as node2' \
-o "$TEMP"/Q154.description.node2.tsv.gz

Concatenate the two description files

In [48]:
!$kgtk cat -i "$TEMP"/Q154.description.node1.tsv.gz -i "$TEMP"/Q154.description.node2.tsv.gz \
-o "$TEMP"/Q154.description.tsv.gz

### Step 7: get the qualifiers

In [49]:
!$kypher -i "$Q154GRAPH" -i qualifiers \
--match 'Q154: ()-[l]->(), qual: (l)-[lq]->(n2)' \
--return 'lq as id, l as node1, lq.label as label, n2 as node2' \
-o "$OUT"/Q154.qualifiers.tsv.gz

In [50]:
result = !zcat < "$OUT"/Q154.qualifiers.tsv.gz 
kgtk_to_dataframe(result)

,id,node1,label,node2
0,P10-P1855-Q7378-555592a4-0-P10-8a982d-0,P10-P1855-Q7378-555592a4-0,P10,"""Elephants Dream (2006).webm"""
1,P10-P2302-Q21502404-d012aef4-0-P1793-f4c2ed-0,P10-P2302-Q21502404-d012aef4-0,P1793,"""(?i).+\\\\.(webm\\|ogv\\|ogg\\|gif)"""
2,P10-P2302-Q21502404-d012aef4-0-P2316-Q21502408-0,P10-P2302-Q21502404-d012aef4-0,P2316,Q21502408
3,P10-P2302-Q21502404-d012aef4-0-P2916-cb0917-0,P10-P2302-Q21502404-d012aef4-0,P2916,"'filename with extension: webm, ogg, ogv, or g..."
4,P10-P2302-Q21510851-5224fe0b-0-P2306-P175-0,P10-P2302-Q21510851-5224fe0b-0,P2306,P175
...,...,...,...,...
93989,Q997294-P421-Q6655-f4ed577c-0-P1264-Q1777301-0,Q997294-P421-Q6655-f4ed577c-0,P1264,Q1777301
93990,Q997294-P421-Q6723-7c4a7768-0-P1264-Q36669-0,Q997294-P421-Q6723-7c4a7768-0,P1264,Q36669
93991,Q997294-P443-cb16c6-94e4e274-0-P407-Q12107-0,Q997294-P443-cb16c6-94e4e274-0,P407,Q12107
93992,Q997294-P485-Q18785452-a7748618-0-P217-2aa283-0,Q997294-P485-Q18785452-a7748618-0,P217,"""1200 E DEPOT"""


### Step 8: consolidate all the files

#### KGTK extensions to Wikidata
KGTK defines extensions to Wikidata, and we want to include those in our graph, so we download them from GitHub:

In [51]:
!wget https://raw.githubusercontent.com/usc-isi-i2/kgtk/dev/kgtk-properties/kgtk.properties.tsv -O "$TEMP"/kgtk.properties.tsv

--2021-01-24 13:11:35--  https://raw.githubusercontent.com/usc-isi-i2/kgtk/dev/kgtk-properties/kgtk.properties.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2617 (2.6K) [text/plain]
Saving to: ‘/Users/pedroszekely/Downloads/kgtk-tutorial/temp/kgtk.properties.tsv’

/Users/pedroszekely 100%[===================>]   2.56K  --.-KB/s    in 0s      

2021-01-24 13:11:40 (12.2 MB/s) - ‘/Users/pedroszekely/Downloads/kgtk-tutorial/temp/kgtk.properties.tsv’ saved [2617/2617]



In [52]:
lines = !cat "$TEMP"/kgtk.properties.tsv 
kgtk_to_dataframe(lines)

,node1,label,node2,id
0,isa,label,"""is a""@en",isa-label-e79b73
1,isa,alias,"""isa""@en",isa-alias-7773c5
2,isa,description,"""Instance or subclass relationship""@en",isa-description-0b5cdc
3,isa,P31,Q18616576,isa-P31-Q18616576
4,isa,P31,Q28326461,isa-P31-Q28326461
5,isa,P31,Q18647519,isa-P31-Q18647519
6,isa,data_type,wikibase-item,isa-data_type-643cc9
7,P279star,label,"""is a""@en",P279star-label-e79b73
8,P279star,alias,"""isa""@en",P279star-alias-7773c5
9,P279star,description,"""Instance or subclass relationship""@en",P279star-description-0b5cdc


#### Property datatype
Wikidata defines data types for properties to specify the type of value expected for each property. We follow this convention, so we include the data types in our KG. 

First take a look at the data types defined for Wikidata properties:

In [53]:
lines = !zcat < "$PROPERTY_DATATYPES" 
kgtk_to_dataframe(lines)

,id,node1,label,node2,node2;wikidatatype,rank
0,P10-datatype,P10,datatype,commonsMedia,,
1,P1001-datatype,P1001,datatype,wikibase-item,,
2,P1003-datatype,P1003,datatype,external-id,,
3,P1004-datatype,P1004,datatype,external-id,,
4,P1005-datatype,P1005,datatype,external-id,,
...,...,...,...,...,...,...
1503,P981-datatype,P981,datatype,external-id,,
1504,P982-datatype,P982,datatype,external-id,,
1505,P984-datatype,P984,datatype,external-id,,
1506,P989-datatype,P989,datatype,commonsMedia,,


We filter this file to select the data types for the properties we use in our graph. We don't care about the datatypes for properties we don't have in our graph:

In [54]:
!$kypher -i "$Q154GRAPH" -i "$PROPERTY_DATATYPES" \
--match 'Q15: (n1)-[]->(), property: (n1)-[l:datatype]->(n2)' \
--return 'distinct l as id, n1 as node1, l.label as label, n2 as node2' \
-o "$TEMP"/Q154.metadata.property.datatype.tsv.gz

In [55]:
lines = !zcat < "$TEMP"/Q154.metadata.property.datatype.tsv.gz 
kgtk_to_dataframe(lines)

,id,node1,label,node2
0,P10-datatype,P10,datatype,commonsMedia
1,P1001-datatype,P1001,datatype,wikibase-item
2,P1004-datatype,P1004,datatype,external-id
3,P1005-datatype,P1005,datatype,external-id
4,P101-datatype,P101,datatype,wikibase-item
...,...,...,...,...
1072,P981-datatype,P981,datatype,external-id
1073,P982-datatype,P982,datatype,external-id
1074,P984-datatype,P984,datatype,external-id
1075,P989-datatype,P989,datatype,commonsMedia


#### Produce the final `all.tsv` file

In [56]:
!$kgtk cat \
-i "$TEMP"/labels.tsv.gz \
-i "$TEMP"/alias.tsv.gz \
-i "$TEMP"/Q154.description.tsv.gz \
-i "$Q154GRAPH" \
-i "$TEMP"/kgtk.properties.tsv \
-i "$TEMP"/Q154.metadata.property.datatype.tsv.gz \
/ compact \
/ sort2 \
-o "$OUT"/all.tsv.gz

Our full alcoholic beverage KG in a pandas dataframe

In [57]:
all = !zcat < "$OUT"/all.tsv.gz 
kgtk_to_dataframe(all)

,id,node1,label,node2
0,P10-P1628-32b85d-7927ece6-0,P10,P1628,"""http://www.w3.org/2006/vcard/ns#Video"""
1,P10-P1628-acf60d-b8950832-0,P10,P1628,"""https://schema.org/video"""
2,P10-P1629-Q34508-bcc39400-0,P10,P1629,Q34508
3,P10-P1659-P1651-c4068028-0,P10,P1659,P1651
4,P10-P1659-P18-5e4b9c4f-0,P10,P1659,P18
...,...,...,...,...
310588,undirected_pagerank-P31-Q47512165,undirected_pagerank,P31,Q47512165
310589,undirected_pagerank-alias-9d4733,undirected_pagerank,alias,"""page rank""@en"
310590,undirected_pagerank-data_type-1a7b30,undirected_pagerank,data_type,quantity
310591,undirected_pagerank-description-ee8b1c,undirected_pagerank,description,"""pagerank canculated on the undirected graph""@en"
